In [1]:
import os
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"  # see issue #152
os.environ["CUDA_VISIBLE_DEVICES"] = "0,1"

In [2]:
import numpy as np
import os
import random

import tensorflow as tf
import matplotlib.pyplot as plt
%matplotlib inline
from tensorflow.contrib import layers, rnn, seq2seq
from tqdm import tqdm

In [3]:
import fasttext as fs

In [4]:
from data_generator import NLUDataGenerator
from tensorflow.contrib.rnn import LSTMCell, LSTMStateTuple
from tensorflow.contrib.legacy_seq2seq import rnn_decoder

In [5]:
TimeMajor = False
batch_size = 32

DataGenDev = NLUDataGenerator('../data/dev/usr_df_final.csv',
                           '../data/ontology_dstc2.json',
                           '../data/slots.txt', None,
                           batch_size = batch_size, time_major=TimeMajor)

DataGen = NLUDataGenerator('../data/train/usr_df_final.csv',
                           '../data/ontology_dstc2.json',
                           '../data/slots.txt', DataGenDev.vocab,
                           batch_size = batch_size, time_major=TimeMajor)

vocab_size = len(DataGen.vocab)
input_embedding_size = 100

len(diff):  175
tmp len:  175
before:  711
fin:  886


In [6]:
x, m1, m2, _lens = next(DataGen)
for x_, m1_, m2_ in zip(x, m1, m2):
    print(DataGen.decode_sentence(x_))
    print(DataGen.decode_acts(m2_))
    print(DataGen.decode_slots(m1_))
    print("-----")

['um', 'thank', 'you', 'goodbye']
['B-bye', 'B-bye', 'B-bye', 'B-bye']
['B-bye', 'B-bye', 'B-bye', 'B-bye']
-----
['in', 'the', 'center', 'part', 'of']
['O', 'O', 'O', 'O', 'O']
['O', 'O', 'O', 'O', 'O']
-----
['i', 'want', 'to', 'find', 'an', 'dontcare', 'restaurant', 'in', 'the', 'east', 'part', 'of', 'town']
['O', 'O', 'O', 'O', 'O', 'B-inform', 'O', 'O', 'O', 'B-inform', 'O', 'O', 'O']
['O', 'O', 'O', 'O', 'O', 'B-pricerange', 'O', 'O', 'O', 'B-area', 'O', 'O', 'O']
-----
['just', 'fuck', 'every', 'time', 'i', 'do', 'it', 'and', 'this', 'moron', 'says', 'that', 'its', 'hungarian', 'sucks']
['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-inform', 'O']
['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-food', 'O']
-----
['how', 'about', 'european', 'type', 'food']
['O', 'O', 'B-inform', 'O', 'O']
['O', 'O', 'B-food', 'O', 'O']
-----
['want', 'something', 'in', 'the', 'centre', 'side', 'of', 'town', 'thats', 'expensive', 'priced']
['O', 'O', 'O',

In [7]:
slots_number = len(DataGen.slots_encode)
acts_number = len(DataGen.acts_encode)
print("slots_number: ", slots_number)
print("acts_number: ", acts_number)
print("vocab_size: ", DataGen.vocab_size)

slots_number:  26
acts_number:  18
vocab_size:  886


In [8]:
class Model:
    
    def __init__(self, train = True, fs_model = None, vocab_size = 886, slots_number = 26, acts_number = 18,
                 batch_size = 32, input_embedding_size = 100, lstm_size = 256, PAD=0):
        tf.reset_default_graph()
        
        def make_embed_mtx(fs_model):
            embedd = np.zeros((vocab_size, input_embedding_size), dtype=np.float32)
            for k, v in DataGen.vocab.items():
                embedd[v] = fs_model[k]
            embedd[0] = [0]*100
            return embedd
        
        self.inputs = tf.placeholder(shape=(batch_size, None), dtype=tf.int32, name='inputs')
        self.mask1 = tf.placeholder(shape=(batch_size, None), dtype=tf.int32, name='tar_mask1')
        self.mask2 = tf.placeholder(shape=(batch_size, None), dtype=tf.int32, name='tar_mask2')
        self.lens = tf.placeholder(shape=(batch_size,), dtype=tf.int32, name='lens')

        if train:
            embedd = make_embed_mtx(fs_model)
            embeddings = tf.get_variable('embeddings', shape=embedd.shape, initializer=tf.constant_initializer(embedd), dtype=tf.float32, trainable=False)
            self.encoded = tf.nn.embedding_lookup(embeddings, self.inputs, name='encoded_train')
        else:
            self.encoded = tf.placeholder(shape=(batch_size, None, input_embedding_size), dtype=tf.float32, name='inference')

        cell = LSTMCell(lstm_size)
        outputs, _ = tf.nn.dynamic_rnn(cell, self.encoded, sequence_length=self.lens, dtype=tf.float32)
        seq_mask = tf.to_float(tf.greater(self.inputs, PAD))

        # regularization:
        #
        # with tf.variable_scope('lm-reg'):
        #     first_words = tf.slice(inputs, [0, 1], [-1, -1])
        #     target = tf.pad(first_words, [[0, 0], [0, 1]])
        #     nwp = layers.fully_connected(outputs, vocab_size, activation_fn=None)
        #     lm_loss = seq2seq.sequence_loss(nwp, target, seq_mask)

        with tf.variable_scope('mask-pred'):
            m1p = layers.fully_connected(outputs, slots_number, activation_fn=None)
            m2p = layers.fully_connected(outputs, acts_number, activation_fn=None)

            self.prediction_m1 = tf.argmax(tf.nn.softmax(m1p), axis = 2, name='prediction_m1')
            self.prediction_m2 = tf.argmax(tf.nn.softmax(m2p), axis = 2, name='prediction_m2')

            self.m1_loss = tf.reduce_sum(seq2seq.sequence_loss(m1p, self.mask1, seq_mask, average_across_batch=False, average_across_timesteps=False), name='m1_loss')
            self.m2_loss = tf.reduce_sum(seq2seq.sequence_loss(m2p, self.mask2, seq_mask, average_across_batch=False, average_across_timesteps=False), name='m2_loss')

        # alternative loss:
        #     log_likelihood_1, transition_params_1 = tf.contrib.crf.crf_log_likelihood(m1p, mask1, lens)
        #     log_likelihood_2, transition_params_2 = tf.contrib.crf.crf_log_likelihood(m2p, mask2, lens)

        lr = tf.Variable(0.0001, trainable=False, name='lr')
        opt = tf.train.GradientDescentOptimizer(lr)
        self.train_op = opt.minimize(self.m1_loss + self.m2_loss, name='train_op')

## Train

In [9]:
fs_model = fs.load_model("./model.bin")
model = Model(fs_model=fs_model)

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    saver = tf.train.Saver()
    avg_loss = -np.ones((3, ))
    alpha = 0.9
    
    pbar = tqdm(range(30000))
    for step in pbar:
        x, m1, m2, _lens = next(DataGen)
        res = sess.run([model.m1_loss, model.m2_loss, model.train_op], {model.inputs: x, model.mask1: m1, model.mask2: m2, model.lens: _lens})
        pbar.set_description(str(res[0]) + ' ' + str(res[1]))
        
#         print(res[0])
#         res = sess.run([lm_loss, m1_loss, m2_loss, train_op], {inputs: x, mask1: m1, mask2: m2, lens: _lens})
#         res = np.array(res[:-1])
        
#         if avg_loss[0] < 0:
#             avg_loss = res
        
#         avg_loss = alpha * avg_loss + (1-alpha) * res
#         pbar.set_description(' '.join('{:.2f}'.format(_) for _ in avg_loss))
        
        
        if(step%1000 == 0):
            
            x_dev, m1_dev, m2_dev, _lens_dev = next(DataGenDev)
            print("step: ", step)
            
            N = 3 # how many examples show to test:
            loss_cv_1, loss_cv_2, _, pred_1, pred_2 = sess.run([model.m1_loss, model.m2_loss, model.train_op, 
                                                                model.prediction_m1, model.prediction_m2], 
                                                    {model.inputs: x_dev, 
                                                      model.mask1: m1_dev,
                                                      model.mask2: m2_dev,
                                                      model.lens: _lens_dev})
            print("CV_loss_slots_1: ", loss_cv_1)
            print("CV_loss_acts_2: ", loss_cv_2)
            
            for x_, m1_, m2_, p1, p2 in zip(x_dev[:N], m1_dev[:N], m2_dev[:N], pred_1[:N], pred_2[:N]):

                print("INPUT: ", DataGen.decode_sentence(x_))
                print("m2_tar: ", DataGen.decode_acts(m2_))
                print("m2_pred: ", DataGen.decode_acts(p2))                
                
                print("-----")
                
                print("m1_tar: ", DataGen.decode_slots(m1_))
                print("m1_pred: ", DataGen.decode_slots(p1))
                
                print("==========================")
    saver.save(sess=sess, save_path='nlu_model_',global_step=30000)

680.369 633.923:   0%|          | 1/30000 [00:00<4:07:23,  2.02it/s]

step:  0
CV_loss_slots_1:  701.7
CV_loss_acts_2:  652.398
INPUT:  ['the', 'price', 'code']
m2_tar:  ['O', 'B-request', 'I-request']
m2_pred:  ['I-confirm', 'I-confirm', 'I-confirm', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']
-----
m1_tar:  ['O', 'B-pricerange', 'I-pricerange']
m1_pred:  ['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']
INPUT:  ['no', 'im', 'looking', 'for', 'pan', 'asian', 'food']
m2_tar:  ['I-reqalts', 'I-reqalts', 'I-reqalts', 'I-reqalts', 'I-reqalts', 'I-reqalts', 'I-reqalts']
m2_pred:  ['B-request', 'B-request', 'B-reqalts', 'B-reqalts', 'B-reqalts', 'B-request', 'B-request', 'O', 'O', 'O', 'O', 'O', 'O', 'O']
-----
m1_tar:  ['I-reqalts', 'I-reqalts', 'I-reqalts', 'I-reqalts', 'I-reqalts', 'I-reqalts', 'I-reqalts']
m1_pred:  ['I-negate', 'I-negate', 'I-negate', 'I-negate', 'I-negate', 'I-negate', 'I-negate', 'O', 'O', 'O', 'O', 'O', 'O', 'O']
INPUT:  ['is', 'it', 'in', 'the', 'centre', 'of', 'town']
m2_tar:  ['O', 'O', 'O', 'O', 'I-

154.311 128.99:   3%|▎         | 1004/30000 [00:26<15:05, 32.02it/s] 

step:  1000
CV_loss_slots_1:  164.516
CV_loss_acts_2:  139.435
INPUT:  ['in', 'the', 'dontcare', 'price']
m2_tar:  ['O', 'O', 'B-inform', 'O']
m2_pred:  ['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']
-----
m1_tar:  ['O', 'O', 'B-pricerange', 'O']
m1_pred:  ['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']
INPUT:  ['addrss', 'and', 'phone', 'number']
m2_tar:  ['B-request', 'O', 'B-request', 'I-request']
m2_pred:  ['O', 'O', 'O', 'B-request', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']
-----
m1_tar:  ['B-addr', 'O', 'B-phone', 'I-phone']
m1_pred:  ['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']
INPUT:  ['how', 'bout', 'dontcare', 'food']
m2_tar:  ['O', 'O', 'B-inform', 'O']
m2_pred:  ['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']
-----
m1_tar:  ['O', 'O', 'B-food', 'O']
m1_pred:  ['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']


162.609 140.55:   7%|▋         | 2007/30000 [00:55<13:34, 34.36it/s] 

...

76.3102 69.357:  93%|█████████▎| 28005/30000 [12:09<01:00, 33.02it/s] 

step:  28000
CV_loss_slots_1:  50.4388
CV_loss_acts_2:  73.248
INPUT:  ['alright', 'give', 'me', 'the', 'addre']
m2_tar:  ['O', 'O', 'O', 'O', 'B-request']
m2_pred:  ['O', 'O', 'O', 'O', 'B-request', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']
-----
m1_tar:  ['O', 'O', 'O', 'O', 'B-addr']
m1_pred:  ['O', 'O', 'O', 'O', 'B-addr', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']
INPUT:  ['is', 'it', 'in', 'the', 'north', 'part', 'of', 'town']
m2_tar:  ['O', 'O', 'O', 'O', 'I-hello', 'O', 'O', 'O']
m2_pred:  ['O', 'O', 'O', 'O', 'B-confirm', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']
-----
m1_tar:  ['O', 'O', 'O', 'O', 'B-area', 'O', 'O', 'O']
m1_pred:  ['O', 'O', 'O', 'O', 'B-area', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']
INPUT:  ['in', 'the', 'south']
m2_tar:  ['O', 'O', 'B-inform']
m2_pred:  ['O', 'O', 'B-inform', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']
-----
m1_tar:  ['O', 'O', 'B-area']
m1_p

23.6942 27.8445:  97%|█████████▋| 29005/30000 [12:33<00:27, 35.63it/s]

step:  29000
CV_loss_slots_1:  87.214
CV_loss_acts_2:  129.687
INPUT:  ['is', 'it', 'in', 'the', 'dontcare', 'of', 'town']
m2_tar:  ['O', 'O', 'O', 'O', 'I-hello', 'O', 'O']
m2_pred:  ['O', 'O', 'O', 'O', 'B-confirm', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']
-----
m1_tar:  ['O', 'O', 'O', 'O', 'B-area', 'O', 'O']
m1_pred:  ['O', 'O', 'O', 'O', 'B-area', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']
INPUT:  ['is', 'it', 'in', 'the', 'dontcare', 'price', 'range']
m2_tar:  ['O', 'O', 'O', 'O', 'I-hello', 'O', 'O']
m2_pred:  ['O', 'O', 'O', 'O', 'B-confirm', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']
-----
m1_tar:  ['O', 'O', 'O', 'O', 'B-pricerange', 'O', 'O']
m1_pred:  ['O', 'O', 'O', 'O', 'B-area', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']
INPUT:  ['i', 'would', 'like', 'a', 'cheap', 'restaurant', 'and', 'it', 'should']
m2_tar:  ['O', 'O', 'O', 'O', 'B-inform', 'O', 'O', 'O', 'O']
m2_pred:  ['O', 'O', 'O', 'O', 'B-inform', 'O', 'O', 'O', '

17.1799 18.3609: 100%|██████████| 30000/30000 [13:00<00:00, 38.46it/s]


In [ ]:
file_writer = tf.summary.FileWriter('../logdir/', sess.graph)

# Использование обученной модели

In [10]:
model = Model(train=False)
fs_model = fs.load_model("./model.bin")

In [11]:
TimeMajor = False
batch_size = 32
DataGenDev = NLUDataGenerator('../data/dev/usr_df_final.csv',
                           '../data/ontology_dstc2.json',
                           '../data/slots.txt', None,
                           batch_size = batch_size, time_major=TimeMajor, digitize = False, fs_model=fs_model)

In [26]:
x_dev, m1_dev, m2_dev, _lens_dev = next(DataGenDev)

In [16]:
with tf.Session() as sess:
    
    new_saver = tf.train.Saver()
    sess.run(tf.global_variables_initializer())
    new_saver.restore(sess, tf.train.latest_checkpoint('./'))

    N = 4
    x_dev, m1_dev, m2_dev, _lens_dev = next(DataGenDev)
    x_dev_vectorized = DataGenDev.vectorize(x_dev, len(m1_dev[0]), embedd_size=100)
    
    loss_cv_1, loss_cv_2, pred_1, pred_2 = sess.run([model.m1_loss, model.m2_loss,
                                                        model.prediction_m1, model.prediction_m2],
                                                   {model.inputs: m1_dev,
                                                      model.encoded:x_dev_vectorized,
                                                      model.mask1: m1_dev,
                                                      model.mask2: m2_dev,
                                                      model.lens: np.array(_lens_dev)})
    
    for x_, m1_, m2_, p1, p2 in zip(x_dev[:N], m1_dev[:N], m2_dev[:N], pred_1[:N], pred_2[:N]):

        print("INPUT: ", x_)
        print("m2_tar: ", DataGenDev.decode_acts(m2_))
        print("m2_pred: ", DataGenDev.decode_acts(p2))                

        print("-----")

        print("m1_tar: ", DataGenDev.decode_slots(m1_))
        print("m1_pred: ", DataGenDev.decode_slots(p1))

        print("==========================")

INFO:tensorflow:Restoring parameters from ./nlu_model_-30000
INPUT:  ['perfect', 'may', 'i', 'have', 'the', 'address']
m2_tar:  ['O', 'O', 'O', 'O', 'O', 'B-request']
m2_pred:  ['O', 'O', 'O', 'O', 'O', 'B-request', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']
-----
m1_tar:  ['O', 'O', 'O', 'O', 'O', 'B-addr']
m1_pred:  ['O', 'O', 'O', 'O', 'O', 'B-addr', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']
INPUT:  ['what', 'is', 'the', 'addre', 'phone', 'and', 'type', 'of', 'food']
m2_tar:  ['O', 'O', 'O', 'B-request', 'B-request', 'O', 'O', 'O', 'B-request']
m2_pred:  ['O', 'O', 'O', 'B-request', 'B-request', 'O', 'O', 'O', 'B-request', 'O', 'O', 'O', 'O', 'O']
-----
m1_tar:  ['O', 'O', 'O', 'B-addr', 'B-phone', 'O', 'O', 'O', 'B-food']
m1_pred:  ['O', 'O', 'O', 'B-addr', 'B-phone', 'O', 'O', 'O', 'B-food', 'O', 'O', 'O', 'O', 'O']
INPUT:  ['what', 'about', 'a', 'polish', 'restaurant', 'in', 'the', 'north', 'of', 'town']
m2_tar:  ['O', 'O', 'O', 'B-inform', 'O', 'O', 'O', 'B-inform', 'O', 'O']
m2_pred

# Метрики качества

In [35]:
## Считаем процент совпавших тегов в одном предложении ##

def acc(tar, pred, lens):
    count = 0
    count_all = 0
    for i in range(lens):
        if (tar[i] != 1):
            count_all+=1
            if (pred[i]==tar[i]):
                count+=1
    if count_all == 0:
        return -1
    return count/count_all

In [36]:
Acc1 = 0
Acc2 = 0

In [37]:
batch_size = 32
with tf.Session() as sess:
    new_saver = tf.train.Saver()
    sess.run(tf.global_variables_initializer())
    new_saver.restore(sess, tf.train.latest_checkpoint('./'))
    
    N_batches = 3000
    incorrect1 = 0 ## предложения, в которых нет значимых тегов1
    incorrect2 = 0 ## предложения, в которых нет значимых тегов2
    
    for i in range(N_batches):
        x_dev, m1_dev, m2_dev, _lens_dev = next(DataGenDev)
        x_dev_vectorized = DataGenDev.vectorize(x_dev, len(m1_dev[0]), embedd_size=100)
        
        loss_cv_1, loss_cv_2, pred_1, pred_2 = sess.run([model.m1_loss, model.m2_loss,
                                                        model.prediction_m1, model.prediction_m2],
                                                    {model.inputs: m1_dev,
                                                      model.encoded:x_dev_vectorized,
                                                      model.mask1: m1_dev,
                                                      model.mask2: m2_dev,
                                                      model.lens: np.array(_lens_dev)})
        sum_batch1 = 0
        sum_batch2 = 0
        
        for tar1, pred1, tar2, pred2, length in zip(m1_dev, pred_1, m2_dev, pred_2, _lens_dev):
            acc1 = acc(tar1, pred1, length)
            acc2 = acc(tar2, pred2, length)
            
            if acc1==-1:
                incorrect1+=1
            else:
                sum_batch1+=acc1
                
            if acc2 == -1:
                incorrect2+=1
            else:
                sum_batch2+=acc2
            
            
        Acc1+=sum_batch1 ## прибавляем сумму за батч
        Acc2+=sum_batch2 ##
    
    print(Acc1/(N_batches*batch_size - incorrect1))
    print(Acc2/(N_batches*batch_size - incorrect2))

INFO:tensorflow:Restoring parameters from ./nlu_model_-30000
0.826976672466895
0.7910093926917128


In [40]:
N_batches*batch_size - incorrect1

85492

In [39]:
incorrect2

10508

### Вывод:

Ну неплохо ж)

## Это чисто черновик, где происходит какая-то фигня с bi-LSTM

In [ ]:
encoder_inputs = tf.placeholder(shape=(None, None), dtype=tf.int32, name='encoder_inputs')
encoder_inputs_length = tf.placeholder(shape=(None,), dtype=tf.int32, name='encoder_inputs_length')

encoder_targets = tf.placeholder(shape=(None), dtype=tf.int32, name='encoder_targets')
embeddings = tf.Variable(tf.random_uniform([vocab_size, input_embedding_size], -1.0, 1.0), dtype=tf.float32)

encoder_inputs_embedded = tf.nn.embedding_lookup(embeddings, encoder_inputs)

In [ ]:
encoder_cell_b = LSTMCell(encoder_hidden_units)
encoder_cell_f = LSTMCell(encoder_hidden_units, reuse=None)

In [ ]:
# encoder_cell = LSTMCell(encoder_hidden_units, reuse=tf.get_variable_scope().reuse)
# encoder_cell_f = LSTMCell(encoder_hidden_units)
((encoder_fw_outputs,
encoder_bw_outputs),
(encoder_fw_final_state,
encoder_bw_final_state)) = (tf.nn.bidirectional_dynamic_rnn(cell_fw=encoder_cell_f,
                                cell_bw=encoder_cell_b,
                                inputs=encoder_inputs_embedded,
                                sequence_length=encoder_inputs_length,
                                dtype=tf.float32, time_major=TimeMajor))
    

In [ ]:
encoder_outputs = tf.concat((encoder_fw_outputs, encoder_bw_outputs), 2)

encoder_final_state_c = tf.concat(
    (encoder_fw_final_state.c, encoder_bw_final_state.c), 1)

encoder_final_state_h = tf.concat(
    (encoder_fw_final_state.h, encoder_bw_final_state.h), 1)

encoder_final_state = LSTMStateTuple(
    c=encoder_final_state_c,
    h=encoder_final_state_h)

In [ ]:
from tensorflow.contrib import layers
next_word_pred = layers.fully_connected(encoder_outputs, vocab_size, ...)



mask1_pred = layers.fully_connected(encoder_outputs, mask1_vocab, ...)
mask2_pred = layers.fully_connected(encoder_outputs, mask2_vocab, ...)


In [ ]:
encoder_final_state_h

In [ ]:
W = tf.Variable(tf.random_uniform([encoder_hidden_units*2, vocab_size], -1, 1), dtype=tf.float32)
b = tf.Variable(tf.zeros([vocab_size]), dtype=tf.float32)

In [ ]:
logits = tf.add(tf.matmul(encoder_final_state_h, W), b)
# encoder_logits = tf.reshape(encoder_logits_flat, (decoder_max_steps, decoder_batch_size, vocab_size))

In [ ]:
logits

In [ ]:
encoder_targets

In [ ]:
losses = tf.nn.sparse_softmax_cross_entropy_with_logits(logits = logits, labels = encoder_targets)